In [1]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'
CONFIG = "../linux-4.14.152/.config"
CONF_FLAG = ["randconfig", "defconfig", "allnoconfig", "allyesconfig", "allmodconfig", "alldefconfig"]

In [2]:
def read_imp_val_csv(file):
    df = pd.read_csv(file)
    df_dict = df.to_dict()
    res = dict()
    for k, v in df_dict['option'].items():
        imp = map(lambda x: x.replace("'", ''), 
                  df_dict['impossible_value'][k][1:-1].split()) 
        res[v] = set(imp)
    return res

In [3]:
impossible_values = read_imp_val_csv('examples/impossible.csv')

In [4]:
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name
            
def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

# Build a dict {option: val} from .config file
def get_dict(filename):
    res = dict()
    with open(filename, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

In [5]:
def get_solution():
    fla = CNF(DIMACS_FLA)
    solu = []
    with Solver(bootstrap_with=fla.clauses) as s:
        s.solve()
    #   keep positive values only
    #   solu = [k for k in s.get_model() if k > 0]
    # A solution of the formula
    return [variables[k] for k in solu if k in variables]

In [59]:
# Dictionary {option : [possible_values+]}
feature = dict()
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c") and '=' in line:
            m = re.search('(c) (\d+) (\w+)=([\w"-/]*)', line)
#             print(m.group(3), m.group(4))
            if m.group(4) != 'nonempty' and m.group(4) != 'n':
                if m.group(3) in feature:
                    feature[m.group(3)].append(m.group(4))
                else:
                    feature[m.group(3)] = [m.group(4)]

In [60]:
import random

# Write .config content using the solution and 
# pick a random value in the list as real value in
# the configuration
def create_file_content(res):
    df_options = pd.read_csv('examples/alloptions-x86.4.14.152.csv')
    # file content
    file = ""
    for elt in res:
        if "CHOICE_" in elt:
            continue
        kconfig_type = search_kconfig_type(elt, df_options)
        if kconfig_type == 'BOOL':
    #         print("CONFIG_{}=y".format(elt))
            file += "CONFIG_{}=y\n".format(elt)
        elif kconfig_type == 'TRISTATE':
            file += "CONFIG_{}=y\n".format(elt)
        elif kconfig_type == "STRING":
            if elt == "IMA_DEFAULT_HASH":
                found = False
                for k in res:
                    if "IMA_DEFAULT_HASH_" in k and k != "IMA_DEFAULT_HASH":
                        file += 'CONFIG_{}="{}"\n'.format(elt, k[17:].lower())
                        found = True
                if not found:
                    file += 'CONFIG_{}=""\n'.format(elt)
            else:
                file += 'CONFIG_{}="{}"\n'.format(elt, random.choice(feature[elt]))
        elif kconfig_type == "INT":
            file += "CONFIG_{}={}\n".format(elt, random.choice(feature[elt]))
        elif kconfig_type == "HEX":
            file += "CONFIG_{}={}\n".format(elt, random.choice(feature[elt]))
    return file

In [61]:
# Create .config (experimentation)
def create_config_xp():
    res = get_solution()
    file_content = create_file_content(res)
    with open('../linux-4.14.152/.config.xp', 'w') as stream:
        stream.write("# EXPERIMENTATION\n##################\n")
        stream.write(file_content)

In [27]:
create_config_xp()

In [62]:
# Compare
conf_dict = get_dict('../linux-4.14.152/.config')
xp_dict = get_dict('../linux-4.14.152/.config.old')

In [70]:
res = "xp; xp_val; conf; conf_val\n"
for k, v in xp_dict.items():
    if k in conf_dict.keys():
        res += "{}; {}; {}; {}\n".format(k, v, k, conf_dict[k])
    else:
        res += "{}; {}; ; \n".format(k, v)
for k, v in conf_dict.items():
    if k not in xp_dict.keys():
        res += "; ; {}; {}\n".format(k, v)
with open("examples/difference.csv", "w") as stream:
    stream.write(res)

In [71]:
df = pd.read_csv("examples/difference.csv", sep=";")
df

,xp,xp_val,conf,conf_val
0,AUXDISPLAY,y,AUXDISPLAY,y
1,KS0108,y,KS0108,y
2,KS0108_DELAY,2,KS0108_DELAY,2
3,W1,y,W1,y
4,HAS_IOMEM,y,HAS_IOMEM,y
...,...,...,...,...
2563,NaN,,ZLIB_DEFLATE,y
2564,NaN,,LZO_COMPRESS,y
2565,NaN,,LZO_DECOMPRESS,y
2566,NaN,,GLOB,y
